In [1]:
import os
import sys
os.chdir('/local/home/dhaziza/entrack')
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="4"
sys.path.append('/local/home/dhaziza/entrack/')

import src.features as ft_def
import csv
import re

def intvals(s):
    vals = re.findall(r'\d+', s)
    return [int(i) for i in vals]

def intval(s):
    vals = intvals(s)
    return vals[0]

def boolval(cond):
    return 1 if cond else 0

def print_stats(dataset, name, field, cond):
    num = 0.0
    total_value = 0.0
    for row in dataset:
        if cond(row):
            num += 1
            total_value += row[field]

    if num == 0:
        print '!! NO DATA FOR %s' % name
        return
    value_mean = total_value/num

    value_std = 0.0
    for row in dataset:
        if cond(row):
            value_std += (row[field] - value_mean)*(row[field] - value_mean)
    value_std /= num
    print '%s Mean %s %f, variance %f [%d entries]' % (name, field, value_mean, value_std, num)

In [2]:
# Koln data
csv_orig = 'data/raw/csv/orig/koln.csv'
csv_output = 'data/raw/csv/koln.csv'

data = []
with open(csv_orig) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        data.append({
            ft_def.AGE: int(row['Alter']),
            ft_def.HEALTH_AD: 0,
            ft_def.HEALTH_EMCI: 0,
            ft_def.HEALTH_LMCI: 0,
            ft_def.HEALTH_MCI: 0,
            ft_def.HEALTH_PD: 1,
            ft_def.HEALTH_SMC: 0,
            ft_def.HEALTHY: 0,
            ft_def.SEX: int(row['Geschlecht (male =1; female =2)'])-1,
            ft_def.STUDY_PATIENT_ID: intval(row['ID'])
        })

with open(csv_output, "wb+") as csvfile:
    writer = csv.DictWriter(csvfile, data[0].keys(), dialect='excel')
    writer.writeheader()
    for row in data:
        writer.writerow(row)

print_stats(data, 'KOLN', 'age', lambda r: True)

KOLN Mean age 63.546875, variance 83.904053 [128 entries]


In [3]:
# ADNI AIBL
csv_orig = 'data/raw/csv/orig/adni_aibl.csv'
csv_output = 'data/raw/csv/adni_aibl.csv'

data = []
ADNI_AIBL_GROUPS = ['Normal', 'AD', 'EMCI', 'LMCI', 'MCI', 'SMC']
with open(csv_orig) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row['Imaging Protocol'] != 'Field Strength=1.5':
            continue
        # Reconstruct a single int subject id from something like '002_S_0295'
        subject_id = intvals(row['Subject ID'])
        subject_id = subject_id[0]*10000 + subject_id[1]
        group = row['DX Group']
        assert(group in ADNI_AIBL_GROUPS)
        data.append({
            ft_def.AGE: intval(row['Age']),
            ft_def.HEALTH_AD: boolval(group == 'AD'),
            ft_def.HEALTH_EMCI: boolval(group == 'EMCI'),
            ft_def.HEALTH_LMCI: boolval(group == 'LMCI'),
            ft_def.HEALTH_MCI: boolval(group == 'MCI'),
            ft_def.HEALTH_PD: 0,
            ft_def.HEALTH_SMC: boolval(group == 'SMC'),
            ft_def.HEALTHY: boolval(group == 'Normal'),
            ft_def.SEX: 0 if row['Sex'] == 'M' else 1,
            ft_def.STUDY_IMAGE_ID: int(row['Image ID']),
            ft_def.STUDY_PATIENT_ID: subject_id,
        })


with open(csv_output, "wb+") as csvfile:
    writer = csv.DictWriter(csvfile, data[0].keys(), dialect='excel')
    writer.writeheader()
    for row in data:
        writer.writerow(row)

print_stats(data, 'ADNI_AIBL', 'age', lambda x: True)
for ft in [ft_def.HEALTH_AD, ft_def.HEALTH_EMCI, ft_def.HEALTH_LMCI, ft_def.HEALTH_MCI, ft_def.HEALTH_SMC, ft_def.HEALTHY]:
    print_stats(data, 'ADNI_AIBL/%s' % ft, 'age', lambda row: row[ft] == 1)

ADNI_AIBL Mean age 76.648725, variance 47.048237 [8862 entries]
ADNI_AIBL/health_ad Mean age 75.707483, variance 57.387601 [1323 entries]
!! NO DATA FOR ADNI_AIBL/health_emci
ADNI_AIBL/health_lmci Mean age 76.335133, variance 51.774314 [2408 entries]
ADNI_AIBL/health_mci Mean age 75.960647, variance 56.531027 [2287 entries]
!! NO DATA FOR ADNI_AIBL/health_smc
ADNI_AIBL/healthy Mean age 77.905415, variance 28.155962 [2844 entries]
